# Notebook 01 — Data Exploration & Preprocessing

**Purpose:** Load the Instacart dataset, explore it, build the 15-category mapping,
create `item_catalog.json` (top 3000 products), and package orders as
`order_baskets.pkl` for Notebooks 02 and 03.

**Input:** Raw CSVs from Kaggle (orders, order_products__prior, products, aisles, departments)

**Output:** `item_catalog.json`, `category_mapping.json`, `order_baskets.pkl`

**Runtime:** ~3–5 min on Kaggle


In [3]:
import os, json, pickle
import pandas as pd
import numpy as np
from collections import Counter
from pathlib import Path

# ── Auto-detect Kaggle vs local ──────────────────────────────────────────────
IS_KAGGLE = os.path.exists("/kaggle/input")
DATA_DIR   = "/kaggle/input/instacart-market-basket-analysis" if IS_KAGGLE else "../data/instacart"
OUTPUT_DIR = "/kaggle/working"                                 if IS_KAGGLE else "../data/output"
MODELS_DIR = "/kaggle/working"                                 if IS_KAGGLE else "../data/models"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
print(f"DATA_DIR   = {DATA_DIR}")
print(f"OUTPUT_DIR = {OUTPUT_DIR}")


DATA_DIR   = ../data/instacart
OUTPUT_DIR = ../data/output


In [15]:
import os

os.path.getsize(f"{DATA_DIR}/order_products__prior.csv") / (1024**2)

550.7952747344971

In [23]:
chunks = pd.read_csv(
    f"{DATA_DIR}/order_products__prior.csv",
    chunksize=1_000_000,   # tune if needed
    low_memory=False
)

order_products = pd.concat(chunks, ignore_index=True)
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [25]:
print("=" * 60)
print("STEP 1: Loading raw data...")
print("=" * 60)

orders          = pd.read_csv(f"{DATA_DIR}/orders.csv")
#order_products  = pd.read_csv(f"{DATA_DIR}/order_products__prior.csv")  -- Very large file , so handled via chunking above.
products        = pd.read_csv(f"{DATA_DIR}/products.csv")
aisles          = pd.read_csv(f"{DATA_DIR}/aisles.csv")
departments     = pd.read_csv(f"{DATA_DIR}/departments.csv")

print(f"Orders:          {len(orders):>12,} rows")
print(f"Order-Products:  {len(order_products):>12,} rows")
print(f"Products:        {len(products):>12,} rows")
print(f"Aisles:          {len(aisles):>12,} rows")
print(f"Departments:     {len(departments):>12,} rows")


STEP 1: Loading raw data...
Orders:             3,421,083 rows
Order-Products:    32,434,489 rows
Products:              49,688 rows
Aisles:                   134 rows
Departments:               21 rows


In [27]:
print("\n" + "=" * 60)
print("STEP 2: Merging datasets...")
print("=" * 60)

products_enriched = (
    products
    .merge(aisles, on="aisle_id", how="left")
    .merge(departments, on="department_id", how="left")
)

order_products_enriched = order_products.merge(
    products_enriched[["product_id", "product_name", "aisle", "department"]],
    on="product_id", how="left"
)

print(f"Enriched order-products: {len(order_products_enriched):,} rows")
print(order_products_enriched.head(3).to_string())



STEP 2: Merging datasets...
Enriched order-products: 32,434,489 rows
   order_id  product_id  add_to_cart_order  reordered           product_name              aisle  department
0         2       33120                  1          1     Organic Egg Whites               eggs  dairy eggs
1         2       28985                  2          1  Michigan Organic Kale   fresh vegetables     produce
2         2        9327                  3          0          Garlic Powder  spices seasonings      pantry


In [29]:
print("\n" + "=" * 60)
print("STEP 3: Data exploration...")
print("=" * 60)

n_orders   = orders["order_id"].nunique()
n_users    = orders["user_id"].nunique()
n_products = products["product_id"].nunique()
print(f"Unique orders:   {n_orders:>12,}")
print(f"Unique users:    {n_users:>12,}")
print(f"Unique products: {n_products:>12,}")
print(f"Avg items/order: {len(order_products) / n_orders:>12.1f}")

product_counts = (
    order_products_enriched
    .groupby("product_name").size()
    .reset_index(name="order_count")
    .sort_values("order_count", ascending=False)
)
print("\nTop 20 most ordered products:")
for _, row in product_counts.head(20).iterrows():
    print(f"  {row['order_count']:>8,}x  {row['product_name']}")



STEP 3: Data exploration...
Unique orders:      3,421,083
Unique users:         206,209
Unique products:       49,688
Avg items/order:          9.5

Top 20 most ordered products:
   472,565x  Banana
   379,450x  Bag of Organic Bananas
   264,683x  Organic Strawberries
   241,921x  Organic Baby Spinach
   213,584x  Organic Hass Avocado
   176,815x  Organic Avocado
   152,657x  Large Lemon
   142,951x  Strawberries
   140,627x  Limes
   137,905x  Organic Whole Milk
   137,057x  Organic Raspberries
   113,426x  Organic Yellow Onion
   109,778x  Organic Garlic
   104,823x  Organic Zucchini
   100,060x  Organic Blueberries
    97,315x  Cucumber Kirby
    89,632x  Organic Fuji Apple
    87,746x  Organic Lemon
    85,020x  Apple Honeycrisp Organic
    84,255x  Organic Grape Tomatoes


In [31]:
print("\n" + "=" * 60)
print("STEP 4: Creating category mapping (aisle → 15 categories)...")
print("=" * 60)

CATEGORY_MAPPING = {
    # DAIRY
    "milk": "dairy", "yogurt": "dairy", "soy lactosefree": "dairy",
    "eggs": "dairy", "cream": "dairy", "butter": "dairy", "cheese": "dairy",
    "refrigerated": "dairy",
    # PRODUCE
    "fresh fruits": "produce", "fresh vegetables": "produce",
    "fresh herbs": "produce", "packaged vegetables fruits": "produce",
    "organic": "produce",
    # MEAT & SEAFOOD
    "meat counter": "meat_seafood", "packaged meat": "meat_seafood",
    "packaged poultry": "meat_seafood", "packaged seafood": "meat_seafood",
    "seafood counter": "meat_seafood", "prepared meat poultry": "meat_seafood",
    "hot dogs bacon sausage": "meat_seafood", "tofu meat alternatives": "meat_seafood",
    # BAKERY
    "bread": "bakery", "baked goods": "bakery", "bakery desserts": "bakery",
    "buns rolls": "bakery", "tortillas flat bread": "bakery",
    "breakfast bakery": "bakery",
    # BEVERAGES
    "water seltzer sparkling water": "beverages", "juice nectars": "beverages",
    "soft drinks": "beverages", "tea": "beverages", "coffee": "beverages",
    "energy sports drinks": "beverages", "drink mixes": "beverages",
    "kombucha": "beverages", "specialty wines champagnes": "beverages",
    "red wines": "beverages", "white wines": "beverages",
    "beers coolers": "beverages", "spirits": "beverages",
    # SNACKS
    "chips pretzels": "snacks", "cookies cakes": "snacks",
    "crackers": "snacks", "granola bars": "snacks",
    "candy chocolate": "snacks", "popcorn jerky": "snacks",
    "nuts seeds dried fruit": "snacks", "trail mix snack mix": "snacks",
    "fruit vegetable snacks": "snacks", "ice cream ice": "snacks",
    "ice cream toppings": "snacks", "bulk dried fruits vegetables": "snacks",
    # FROZEN
    "frozen meals": "frozen", "frozen pizza": "frozen",
    "frozen appetizers sides": "frozen", "frozen breakfast": "frozen",
    "frozen dessert": "frozen", "frozen produce": "frozen",
    "frozen meat seafood": "frozen", "frozen vegan vegetarian": "frozen",
    "frozen juice": "frozen",
    # HOUSEHOLD
    "paper goods": "household", "cleaning products": "household",
    "laundry": "household", "trash bags liners": "household",
    "dish detergents": "household", "air fresheners candles": "household",
    "cat food care": "household", "dog food care": "household",
    "food storage": "household", "plates bowls cups flatware": "household",
    "kitchen supplies": "household", "more household": "household",
    # PERSONAL CARE
    "skin care": "personal_care", "hair care": "personal_care",
    "oral hygiene": "personal_care", "deodorants": "personal_care",
    "shaving needs": "personal_care", "bath body wash": "personal_care",
    "facial care": "personal_care", "feminine care": "personal_care",
    "first aid": "personal_care", "eye ear care": "personal_care",
    "vitamins supplements": "personal_care", "baby bath body care": "personal_care",
    "baby food formula": "personal_care", "diapers wipes": "personal_care",
    "digestion": "personal_care", "cold flu allergy": "personal_care",
    "pain relief heat pads": "personal_care", "beauty": "personal_care",
    "baby accessories": "personal_care",
    # CONDIMENTS
    "condiments": "condiments", "oils vinegars": "condiments",
    "salad dressing toppings": "condiments",
    "marinades meat preparation": "condiments",
    "pickled goods olives": "condiments",
    "spices seasonings": "condiments",
    # GRAINS & PASTA
    "pasta": "grains_pasta", "dry goods pasta": "grains_pasta",
    "grains rice dried goods": "grains_pasta",
    "bulk grains rice dried goods": "grains_pasta",
    "pasta sauce": "grains_pasta", "asian foods": "grains_pasta",
    "latino foods": "grains_pasta", "indian foods": "grains_pasta",
    "kosher foods": "grains_pasta", "baking ingredients": "grains_pasta",
    "baking supplies decor": "grains_pasta",
    # CANNED GOODS
    "canned meals beans": "canned_goods", "canned meat seafood": "canned_goods",
    "canned jarred vegetables": "canned_goods",
    "canned fruit applesauce": "canned_goods",
    "soup broth bouillon": "canned_goods", "packaged produce": "canned_goods",
    # BREAKFAST
    "cereal": "breakfast", "breakfast bars pastries": "breakfast",
    "pancake waffle mixes": "breakfast", "granola": "breakfast",
    "spreads": "breakfast", "honeys syrups nectars": "breakfast",
    # DELI
    "deli": "deli", "prepared soups salads": "deli", "lunch meat": "deli",
    "prepared meals": "deli", "specialty cheeses": "deli",
    "fresh pasta": "deli",
    # OTHER
    "missing": "other", "other": "other",
}

# Auto-map any unmapped aisles to "other"
all_aisles = set(aisles["aisle"].str.lower().values)
mapped    = set(k.lower() for k in CATEGORY_MAPPING)
unmapped  = all_aisles - mapped
if unmapped:
    print(f"Note: {len(unmapped)} unmapped aisles → mapped to 'other':")
    for a in sorted(unmapped):
        print(f"  - {a}")
    for a in unmapped:
        CATEGORY_MAPPING[a] = "other"

products_enriched["category"] = (
    products_enriched["aisle"].str.lower()
    .map(CATEGORY_MAPPING).fillna("other")
)

cat_dist = products_enriched["category"].value_counts()
print("\nProducts per category:")
for cat, count in cat_dist.items():
    print(f"  {count:>6}  {cat}")

with open(f"{OUTPUT_DIR}/category_mapping.json", "w") as f:
    json.dump(CATEGORY_MAPPING, f, indent=2)
print(f"\nSaved: {OUTPUT_DIR}/category_mapping.json")



STEP 4: Creating category mapping (aisle → 15 categories)...
Note: 19 unmapped aisles → mapped to 'other':
  - body lotions soap
  - cocoa drink mixes
  - doughs gelatins bake mixes
  - dry pasta
  - energy granola bars
  - fresh dips tapenades
  - frozen breads doughs
  - hot cereal pancake mixes
  - instant foods
  - mint gum
  - muscles joints pain relief
  - other creams cheeses
  - packaged cheese
  - poultry counter
  - preserved dips spreads
  - protein meal replacements
  - refrigerated pudding desserts
  - shave needs
  - soap

Products per category:
    8453  other
    6367  snacks
    5920  personal_care
    4521  beverages
    4057  household
    3009  condiments
    2835  frozen
    2813  grains_pasta
    2673  dairy
    2124  canned_goods
    1652  produce
    1534  breakfast
    1516  bakery
    1230  deli
     984  meat_seafood

Saved: ../data/output/category_mapping.json


In [33]:
print("\n" + "=" * 60)
print("STEP 5: Building item catalog (top 3000 products)...")
print("=" * 60)

product_freq = (
    order_products_enriched
    .groupby(["product_id", "product_name", "aisle", "department"])
    .size().reset_index(name="order_count")
    .sort_values("order_count", ascending=False)
)
product_freq.to_csv(f"{OUTPUT_DIR}/product_frequency.csv", index=False)

TOP_N = 3000
top_products = product_freq.head(TOP_N).copy()
top_products["category"] = (
    top_products["aisle"].str.lower()
    .map(CATEGORY_MAPPING).fillna("other")
)

CATEGORY_UNITS = {
    "dairy":         ["liters", "pieces", "cartons", "bottles"],
    "produce":       ["kg", "pieces", "bunches", "bags"],
    "meat_seafood":  ["kg", "pieces", "packs"],
    "bakery":        ["loaves", "pieces", "packs"],
    "beverages":     ["bottles", "liters", "cans", "packs"],
    "snacks":        ["packs", "bags", "pieces", "boxes"],
    "frozen":        ["packs", "boxes", "bags"],
    "household":     ["pieces", "packs", "rolls", "bottles"],
    "personal_care": ["pieces", "bottles", "tubes", "packs"],
    "condiments":    ["bottles", "jars", "pieces"],
    "grains_pasta":  ["packs", "kg", "bags", "boxes"],
    "canned_goods":  ["cans", "jars", "pieces"],
    "breakfast":     ["boxes", "packs", "jars"],
    "deli":          ["kg", "pieces", "packs", "slices"],
    "other":         ["pieces", "packs"],
}

item_catalog = [
    {
        "name":         row["product_name"],
        "name_lower":   row["product_name"].lower(),
        "category":     row["category"],
        "common_units": CATEGORY_UNITS.get(row["category"], ["pieces"]),
        "avg_price":    None,
        "is_seasonal":  False,
        "peak_months":  [],
        "order_count":  int(row["order_count"]),
    }
    for _, row in top_products.iterrows()
]

with open(f"{OUTPUT_DIR}/item_catalog.json", "w") as f:
    json.dump(item_catalog, f, indent=2)
print(f"Saved: {OUTPUT_DIR}/item_catalog.json  ({len(item_catalog)} items)")

cat_counts = Counter(item["category"] for item in item_catalog)
print("\nCatalog category breakdown:")
for cat, count in sorted(cat_counts.items(), key=lambda x: -x[1]):
    print(f"  {count:>5}  {cat}")



STEP 5: Building item catalog (top 3000 products)...
Saved: ../data/output/item_catalog.json  (3000 items)

Catalog category breakdown:
    484  dairy
    440  produce
    360  other
    354  snacks
    227  beverages
    198  frozen
    165  bakery
    137  canned_goods
    123  breakfast
    111  grains_pasta
    100  condiments
     85  meat_seafood
     80  deli
     70  household
     66  personal_care


In [35]:
print("\n" + "=" * 60)
print("STEP 6: Building order baskets...")
print("=" * 60)

catalog_names = set(item["name"] for item in item_catalog)
op_filtered = order_products_enriched[
    order_products_enriched["product_name"].isin(catalog_names)
]
print(f"Order-product rows after filtering to catalog: {len(op_filtered):,}")

baskets = (
    op_filtered
    .sort_values(["order_id", "add_to_cart_order"])
    .groupby("order_id")["product_name"]
    .apply(list).values.tolist()
)
baskets = [b for b in baskets if len(b) >= 2]
print(f"Total baskets: {len(baskets):,}")
print(f"Avg basket size: {np.mean([len(b) for b in baskets]):.1f}")

with open(f"{OUTPUT_DIR}/order_baskets.pkl", "wb") as f:
    pickle.dump(baskets, f)
print(f"Saved: {OUTPUT_DIR}/order_baskets.pkl")

print("\nSample baskets:")
for i, b in enumerate(baskets[:3]):
    print(f"  Order {i+1}: {b[:5]}{'...' if len(b) > 5 else ''}")



STEP 6: Building order baskets...
Order-product rows after filtering to catalog: 23,716,660
Total baskets: 2,849,883
Avg basket size: 8.2
Saved: ../data/output/order_baskets.pkl

Sample baskets:
  Order 1: ['Organic Egg Whites', 'Michigan Organic Kale', 'Garlic Powder', 'Carrots', 'All Natural No Stir Creamy Almond Butter']
  Order 2: ['Total 2% with Strawberry Lowfat Greek Strained Yogurt', 'Unsweetened Almondmilk', 'Lemons', 'Organic Baby Spinach', 'Unsweetened Chocolate Almond Breeze Almond Milk']...
  Order 3: ['Plain Pre-Sliced Bagels', 'Chewy 25% Low Sugar Chocolate Chip Granola', 'Oats & Chocolate Chewy Bars', 'Nutri-Grain Soft Baked Strawberry Cereal Breakfast Bars', 'Tiny Twists Pretzels']...


In [37]:
print("\n" + "=" * 60)
print("CHECKPOINT: Notebook 01 Complete")
print("=" * 60)
print(f"""
Files created in {OUTPUT_DIR}:
  category_mapping.json  ({len(CATEGORY_MAPPING)} aisle → category entries)
  item_catalog.json      ({len(item_catalog)} items)
  product_frequency.csv  (all products ranked by order count)
  order_baskets.pkl      ({len(baskets):,} order baskets)

Next → Notebook 02 (Apriori co-purchase rules)
""")



CHECKPOINT: Notebook 01 Complete

Files created in ../data/output:
  category_mapping.json  (148 aisle → category entries)
  item_catalog.json      (3000 items)
  product_frequency.csv  (all products ranked by order count)
  order_baskets.pkl      (2,849,883 order baskets)

Next → Notebook 02 (Apriori co-purchase rules)

